# Co to MAC?
Message Authentication Code to jednokierunkowa funkcja wykorzystująca klucz tajny w celu utworzenia skrótu wiadomości. Kody uwierzytelnienia wiadomości wykorzystywane są do uwierzytelnienia danych oraz zapewnienia ich integralności. Od klasycznych funkcji jednokierunkowych odróżnia je to, że poprawność wiadomości mogą sprawdzić tylko osoby dysponujące kluczem tajnym.<br>
<br>
Przykład użycia MAC podczas komunikacji między dwoma użytkownikami:
<div><img src='resources/MAC.png'></div>
<br><br>


## Zastosowania MAC
1. Weryfikacja adresów e-mail podczas zakładania konta.
2. E-mail dotyczący resetowania hasła.
3. Komunikacja z weryfikacją dwuetapową.
4. W komunikacji przy pomocy protokołów internetowych SSL, SSH, IPsec.

# HMAC
HMAC (Hash-based Message Authentication Code) jest typem MAC, który powstaje po zastosowaniu funkcji haszującej na wiadomości wymagającej uwierzytelnienia i kluczu tajnym. HMAC wykorzystuje między innymi funkcję haszujące takie jak MD-5, SHA-1, SHA-256. HMAC działa bardzo podobnie do podpisów cyfrowych, ale w przeciwieństwie do nich wykorzystuje klucz symetryczny.
## Działanie
#### Funkcje i zmienne pomocnicze oraz dane wejściowe:<br>
- klucz - tajny klucz dodawany do wiadomości<br>
- dane - dane wymagające uwierzytelnienia<br>
- hash() - wykorzystywana funkcja skrótu<br>
- pad() - funckja, która dopełnia zmienną zerowymi bitami do rozmiaru bloku<br>
- blokS - rozmiar bloku wykorzystywany przez funkcje skrótu<br>
- || - konkatenacja dwóch łańcuchów znaków<br>
- ⊕ - XOR<br>
- opad - zewnętrzne dopełnienie, składa się z bitów o wartości 0x5c powtarzanych do rozmiaru blokS<br>
- ipad - wewnętrzne dopełnienie, składa się z bitów o wartości 0x36 powtarzanych do rozmiaru blokS<br><br>
#### Pseudokod
<em>
if (dlugosc_klucza > blokS)<br>
&emsp; klucz = hash(klucz)<br><br>
if (dlugosc_klucza < blokS)<br>
&emsp; klucz = pad(klucz, blokS)<br><br>
opad_klucz = klucz ⊕ (opad * blokS)<br>
ipad_klucz = klucz ⊕ (ipad * blokS)<br><br>
return hash(opad_klucz || hash(ipad_klucz || dane))<br><br>
</em>
<img src="resources/hmac_schemat.png">

In [16]:
import hashlib

def XOR(x, y):
    return bytes(x[i] ^ y[i] for i in range(min(len(x), len(y))))

def OR(x, y):
    return bytes(x[i] | y[i] for i in range(min(len(x), len(y))))

def hmac_sha256(key_K, data):
    # Zmiana długości klucza, tak aby odpowiadał rozmiarowi bloku
    if len(key_K) > 512:
        key_K = hashlib.sha256(key_K).digest()
    padded_K = key_K + b'\x00' * (512 - len(key_K))

    # Tworzenie podkluczy przy pomocy stałych opad i ipad
    ipad_K = XOR(padded_K, b'\x36' * 512)
    opad_K = XOR(padded_K, b'\x5c' * 512)

    # Generowanie sumy kontrolnej wiadomości
    hash = hashlib.sha256(opad_K + hashlib.sha256(ipad_K + data).digest())
    return hash.digest()

k = b'2e80b22d8c7bf6b2bc049f57f69d01a74b2e8f49f6f41a2324f65c6ecc50c100783df82b3e92ae9f791fa14af415ebf2036294200c1240dbfda8538eac70cb0e'
data = b"Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas eros nibh, interdum."
result = hmac_sha256(k, data)
print("Klucz: ", k)
print("Wiadomość: ", data)
print("Wartość sumy kontrolnej: ", result.hex())

Klucz:  b'2e80b22d8c7bf6b2bc049f57f69d01a74b2e8f49f6f41a2324f65c6ecc50c100783df82b3e92ae9f791fa14af415ebf2036294200c1240dbfda8538eac70cb0e'
Wiadomość:  b'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Maecenas eros nibh, interdum.'
Wartość sumy kontrolnej:  27acf3e227ed5fc0bca238f5f4358d26dcb5b90aefc078907f977b25a59629fa


# CBC-MAC

CBC-MAC jest algorytmem pozwalającym na obliczenie MAC przy użyciu szyfru blokowego (np. AES) w trybie Cipher block chaining (CBC).

### <em>Ci = E(Mi⊕C(i-1))</em>
- M - wiadomość
- Mi - i-ty blok wiadomości
- E - szyfr blokowy
- Ci - i-ty blok szyfrogramu
- C0 = 0

### CBC-MAC
<div>
<img src="resources/CBC-MAC.png" width="700"/>
</div>

- mx - Kolejny blok wiadomości<br>
- k - klucz<br>
- E - szyfr blokowy<br>

CBC-MAC wykorzystuje tryb CBC szyfrowania blokowego, z dwoma różnicami:
- rezultatem CBC-MAC jest tylko ostatni blok szyfrogramu
- Wektor inicjalizacji (IV) jest równy 0

### CBC

<div>
<img src="resources/CBC.png" width="700"/>
</div>



In [2]:
import AES
import random

BLOCK_SIZE = 128

def zero_padding(message,nearest_multiple):
    message_length = len(message)
    n=0
    while(n*nearest_multiple<message_length):
        n+=1

    target_length = n*nearest_multiple
    while(len(message)<target_length):
        message+='0'

    return message

def generate_random_binary(length):

    result = ''
    for x in range(length):
        result+=str(random.randint(0,1))

    return result

def generate_zeros(length):
    return '0'*length

message = generate_random_binary(BLOCK_SIZE*10+24)
initialization_vector = generate_zeros(BLOCK_SIZE)
key = generate_random_binary(BLOCK_SIZE)

def XOR(string1, string2):
    
    if(len(string1)!=len(string2)):
        return -1

    xored = ''
    for i in range(len(string1)):
        if string1[i]==string2[i]:
            xored+='0'
        else:
            xored+='1'

    return xored

def parse_into_blocks(message,block_size):
    chunked_message = [message[i:i+block_size] for i in range(0, len(message),block_size)]
    return chunked_message

def CBC_STEP(block, iv, key):
    xored = XOR(block, iv)
    return AES.AES_encrypt(xored,key)

def CBC_MAC(message, iv, key):

    global BLOCK_SIZE
    chunked_message = parse_into_blocks(message, BLOCK_SIZE)
    length = len(chunked_message)

    current_step = CBC_STEP(chunked_message[0], iv, key)
    for i in range(1,length):
        current_step = CBC_STEP(chunked_message[i], current_step, key)

    return current_step

print("Wiadomość: ",message,"\n")
print("Klucz: ",key,"\n")
print("Skrót: ",CBC_MAC(zero_padding(message,128), initialization_vector, key))


Wiadomość:  0100011011010110110001110110011101010110000010110110010011111101010100000010000111101010110000101000100111101110001011111100110110110110111001100011110100010100110011011101100111111110111100110001110100000111100011011111111001001000110101011000100111110001101100010110000101101110110001110111001111001010110101101010111100111110100001100000011010010101100111010000101001001111110010100101110111000001011010100110110111110110101110101010100101011010111111000110001101011011001100110000100000101010101011000011001011100011110110010011100000011001000100100010010110111101100010100001110100001001001000111001011101000001000100110110010011110010011100000001000100000001011101100110001001001000101111011000001011101000101000111001011000010010010110000000100110001001000000011100010110011101010001100010100011101100101000100000110100110011000010010101011011000100100110011110110111011000000011110000001010000000111100111100111011111101110100001011001101011101110101110010101000000110010101010010


### Bezpieczeństwo
Możliwość szyfrowania wiadomości o dowolnej długości prowadzi do powstania podatności:<br>
Mając dwie pary (wiadomość, skrót): (M1,H1), (M2,H2) można utworzyć trzecią wiadomość o skrócie H2.
Do tego celu należy XORować pierwszy blok M2 z H1. W rezultacie:<br><em>
- M3 = M1 || (M2[0] ⊕ H1) || M2[1:]
- CBC_MAC(M2) == CBC_MAC(M3)</em>

Aby wyeliminować ten problem powstały dwa rozwiązania:
- Length prepending
- Encrypt last block

#### Length prepending
Rozwiązanie 'Length Prepending' polega na umieszczeniu informacji o długości wiadomości w pierwszym bloku. Eliminuje to wyżej opisany problem, jednak może okazać się uciążliwe jeżeli nie znamy długości wiadomości.

#### Encrypt Last Block
Rozwiązanie ECBC-MAC polega na zaszyfrowaniu rezultatu CBC-MAC przy użyciu innego klucza.<br>

Następnym zagrożeniem dla bezpieczeństwa funkcji CBC-MAC jest zezwolenie na użycie dowolnego wektora inicjalizacji. Otwiera to furtkę do preparowania wiadomości. Jako, że operacja XOR jest przemienna, możnaby stworzyć 2 pary (wiadomość, IV): Pierwsza para to dowolna wiadomość i dowolny wektor, a druga para składa się z tej samej wiadomości i tego samego wektora z zamienionymi odpowiadającymi sobie dowolnymi bitami. W związku z przemiennością XOR, skróty MAC tych dwóch wiadomości będą takie same. Już rezultat operacji XOR na pierwszym bloku i wektorze inicjalizacji będzie taki sam w obu przypadkach.

<b>Przykład:</b><br>
'00110101' ⊕ '10111010' = '10110000'<br>
Po zamianie pierwszych bitów wiadomości:<br>
'10110101' ⊕ '00111010' = '10110000'<br>
<br>
Wynik obu operacji XOR jest taki sam.


### Przykład preparowania skrótów z sekcji 'Bezpieczeństwo'

In [3]:
BLOCK_SIZE = 128

key = generate_random_binary(BLOCK_SIZE)
message_1 = generate_random_binary(BLOCK_SIZE*10)
message_2 = generate_random_binary(BLOCK_SIZE*10)
hash_1 = CBC_MAC(message_1,initialization_vector,key)
hash_2 = CBC_MAC(message_2,initialization_vector,key)
xored_block = XOR(message_2[0:BLOCK_SIZE],hash_1) # XOR pierwszego bloku message_2 i skrótu hash_1
forged_message = message_1 + xored_block + message_2[BLOCK_SIZE:]
hash_forged = CBC_MAC(forged_message,initialization_vector,key)
print("Skrót oryginalnej drugiej wiadomości: ",hash_2)
print("Skrót spreparowanej przez nas wiadomości: ",hash_forged)

Skrót oryginalnej drugiej wiadomości:  01001110011011100000101000101111001101011010101001010010010011100110110011001010101101110000001111010100010111101010010111001101
Skrót spreparowanej przez nas wiadomości:  01001110011011100000101000101111001101011010101001010010010011100110110011001010101101110000001111010100010111101010010111001101


### Ten sam przykład, z zastosowaniem Length-Prepending

In [4]:
# LENGTH PREPENDING

def int_to_str(integer, length): # Funkcja zamieniająca liczbę Base10 na ciąg zer i jedynek o długości "length"
    if integer==0:
        return '0'*length
    reszty = []
    while(integer!=1):
        reszta = integer%2
        reszty.append(str(reszta))
        integer=integer//2
    reszty.append('1')
    while(len(reszty)<length):
        reszty.append('0')
    reszty.reverse()

    return "".join(reszty)

def length_prepending_CBC_MAC(message, iv, key):

    global BLOCK_SIZE

    message_length = len(message)/BLOCK_SIZE # Długość wiadomości w blokach
    message_length = int_to_str(int(message_length),BLOCK_SIZE)
    message = message_length + message
    return CBC_MAC(message,iv,key)

# key = generate_random_binary(BLOCK_SIZE)
# message_1 = generate_random_binary(BLOCK_SIZE*10)
# message_2 = generate_random_binary(BLOCK_SIZE*10)
hash_1 = length_prepending_CBC_MAC(message_1,initialization_vector,key)
hash_2 = length_prepending_CBC_MAC(message_2,initialization_vector,key)
xored_block = XOR(message_2[0:BLOCK_SIZE],hash_1) # XOR pierwszego bloku message_2 i skrótu hash_1
forged_message = message_1 + xored_block + message_2[BLOCK_SIZE:]
hash_forged = length_prepending_CBC_MAC(forged_message,initialization_vector,key)
print("Skrót oryginalnej drugiej wiadomości: ",hash_2)
print("Skrót spreparowanej przez nas wiadomości: ",hash_forged)

Skrót oryginalnej drugiej wiadomości:  10111010110101011111110101000011001100100111010100000110100100000100001111000000001000010101100011001001110001001010001110001011
Skrót spreparowanej przez nas wiadomości:  01001110011011100000101000101111001101011010101001010010101101000110110011001010101101110000001111010100010111101010010111001101


# CMAC
CMAC (Cipher-based Message Authentication Code) czasami nazywany One-key MAC1 w sposobie działania jest bardzo zbliżony do CBC-MAC, różnica następuje przy operacji na ostatnim bloku. Suma kontrolna ma długość bloku.
## Działanie
|| - konkatenacja dwóch łańcuchów znaków
⊕ - XOR
1. Generowany jest tymczasowy klucz k0 = AES(00...0).
2. Jeśli najbardziej wysuniętym na lewo bitem k0 jest 0 wtedy pierwszy podklucz k1 = k0 << 1 w przeciwnym wypadku k1 = (k0 << 1) ⊕ C, gdzie C jest stałą zależną od rozmiaru bloku (0x1B dla 64, 0x87 dla 128 i 0x425 dla 256).
3. Jeśli najbardziej wysuniętym na lewo bitem k1 jest 0 wtedy drugi podklucz k2 = k1 << 1 w przeciwnym wypadku k2 = (k1 << 1) ⊕ C.
4. Wiadomość dzielona jest bloki równych długości zależnej od rozmiaru bloku na którym operują funkcja szyfrująca. Ostatni blok nie musi być kompletnym blokiem.
5. Jeśli ostatni blok m<sub>n</sub> jest kompletny wtedy m<sub>n</sub>' = k1 ⊕ m<sub>n</sub> w przeciwnym wypadku m<sub>n</sub>' = k2 ⊕ (m<sub>n</sub> || 10...0).
6. Niech startowy blok wynosi c<sub>0</sub> = 00...0.
7. Dla bloków od 1 do n-1 oblicz c<sub>i</sub> = AES(c<sub>i-1</sub> ⊕ m<sub>i</sub>).
8. Ostatni blok który również będzie sumą kontrolną jest równy c<sub>n</sub> = AES(c<sub>n-1</sub> ⊕ m<sub>n</sub>').
<br>

<img src="resources/cmac_schemat.jpg">

In [5]:
from lib_cmac import AES_encrypt, XOR, LEFTSHIFT, OR
from textwrap import wrap

# Opis funkcji pomocniczych
# AES_encrypt - implementacja AES dla bloków 128-bitowy
# OR, XOR, LEFTSHIFT - podstawowe operatory bitowe

def CMAC_128(message, key):
    C = '10000111100001111000011110000111100001111000011110000111100001111000011110000111100001111000011110000111100001111000011110000111'
    blocks = wrap(message, 128)
    
    # Generowanie podkluczy
    k0 = AES_encrypt('0'*128, key)
    if(k0[0] == '0'):
        k1 = LEFTSHIFT(k0, 1)
    else:
        k1 = XOR(LEFTSHIFT(k0, 1), C)
    
    if(k1[0] == '0'):
        k2 = LEFTSHIFT(k1, 1)
    else:
        k2 = XOR(LEFTSHIFT(k1, 1), C)
        
    # Operacje na ostatnim bloku wiadomośći
    if(len(blocks[-1]) == 128):
        last_block = XOR(k1, blocks[-1])
    else:
        pad = '1' + (127-len(blocks[-1]))*'0'
        pad = blocks[-1] + pad
        last_block = XOR(k2,pad)

    # Operacje na blokach od 1 do n-1
    crypted = AES_encrypt(XOR(128*'0', blocks[0]), key)
    for i in range(1, len(blocks)-1):
        crypted = AES_encrypt(XOR(crypted, blocks[i]), key)
    
    # Generowanie sumy kontrolnej wiadomości
    tag = AES_encrypt(XOR(crypted, last_block), key)

    return tag

# tekst 232-bity 
bin_plaintext = '0100101101110010011110010111000001110100011011110110011101110010011000010110011001101001011000010010000001001011011011110110010001111001001000000110000101110101011101000110111101110010011110010111101001100001011000110110101001101001' #"00100000000100001000000000000010000000000000000100100011010000110010010000000001010101000101000101010100001000000100000000010000000000000001000000000100001001000110100001100100100000000010101010001010001010101000011010100010011010001010001101010001001101000101000100000000000000000000000000000000000000000001001000110100001100100100000000010101010001010001010101000011010100010011010001010001"
# klucz 128-bitowy 
bin_key =       "00000000000001111111100010001111111110001000111111111000100011111111100011111111000100011111111100010001111111110001000111111111"


print("Wiadomość: ", bin_plaintext, '\n')
print("Klucz: ", bin_key, '\n')
print("Suma kontrolna: ", CMAC_128(bin_plaintext, bin_key))

Wiadomość:  0100101101110010011110010111000001110100011011110110011101110010011000010110011001101001011000010010000001001011011011110110010001111001001000000110000101110101011101000110111101110010011110010111101001100001011000110110101001101001 

Klucz:  00000000000001111111100010001111111110001000111111111000100011111111100011111111000100011111111100010001111111110001000111111111 

Suma kontrolna:  01010100011010110011000101000010011010100100101101100000101000100101111110110000001011110011110000111000011100001000010001101010


# UMAC
UMAC (Universal Message Authentication Code) wykorzystuje uniwersalne haszowanie do generowania sum kontrolnych. Uniwersalna funkcja haszująca tworzy skróty wybierając losowo funkcje haszującą z rodziny. Gwarantuje to niską liczbę kolizji i wysoką wydajnośc. W poniższej implementacji stosowana jest 3 warstwowa funkcja haszująca.
<img src="resources/umac_schemat.png">

In [6]:
from lib_umac import add_32, add_64, multiply_64, uint2str, endianswap, zeropad, str2uint, AND, XOR, kdf, pdf
from textwrap import wrap
import math

# Opis funkcji pomocniczych
# AND, XOR - podstawowe operatory bitowe
# zeropad(S, n) - dopełniania stringa 0, tak aby jego długość w bitach była podzielna przez 8*n
# prime - wypisuje największą liczbę pierwszą mniejszą od podanej
# str2uint - konwertuje stringa na liczbę całkowitą
# uint2str(n,i) - konwertuje liczbę całkowitą na łańcuch bitów o długości i w bajtach
# add_32, add_64, multiply_64 - dodawanie i mnożenie binarne dla liczb 32 i 64 bajtowych
# endianswap - zmienia kolejność bajtów z Big Endian na Little Endian w celu przyspieszenia
#              generowania sum kontrolnych dla systemów Little Endian
# kdf(key,index,length) - funkcja generująca podkluczy o podanej długości, pole index jest dodatnią liczbą całkowitą,
#                         której celem jest zapewnienie pseudolosowości podkluczy
# pdf - funkcja generująca pseudolosowe dopełnienie

## Warstwa pierwsza
Pierwsza warstwa haszowania dzieli wiadomość na 1024-bajtowe bloki i haszuje każdego z nich funkcją <em><b>NH</em></b>.
<br><br>
<em><b>L1_Hashing</em></b>:<br>
Przed wysłaniem bloku do funkcji <em><b>NH</em></b> dokonywany jest na nim <em><b>endian swap</em></b>.
1. Dzieli wiadomość na 1024 bajtowe fragmenty.
2. Tworzy zmienną 64-bitową zmienną <em>Len</em> o wartości 8192.
3. Dla pełnych bloków dodaje binarnie wynik funkcji <em><b>NH</em></b> dla danego bloku i zmienną <em>Len</em>.
4. Zmiana wartości zmiennej <em>Len</em> na długość ostatiego bloku w bitach.
5. Dla ostatniego bloku dodaje binarnie blok dopełniony do 32 bajtów i zmienną <em>Len</em>.<br><br>

<em><b>NH</em></b>:
1. Dzieli wiadomość i klucz na 32-bitowe fragmenty.
2. Deklaruje zmienną początkową <em>Y</em> równą 0.
3. W pętli wykonuje poniższy schemat zwiększając o 8 wartość na koniec każdej pętli dopóki nie przekroczy on długości wiadomości podzielonej przez 32. <br><br>
dla początkowej wartości iteratora i=0 <br><em>
Y = Y +_64 ((Message[i+0] +_32 Key[i+0]) *_64 (Message[i+4] +_32 Key[i+4]))<br>
Y = Y +_64 ((Message[i+1] +_32 Key[i+1]) *_64 (Message[i+5] +_32 Key[i+5]))<br>
Y = Y +_64 ((Message[i+2] +_32 Key[i+2]) *_64 (Message[i+6] +_32 Key[i+6]))<br>
Y = Y +_64 ((Message[i+3] +_32 Key[i+3]) *_64 (Message[i+7] +_32 Key[i+7]))<br></em>

In [7]:
def nh(key, message):
    # Przygotowanie zmiennych i podział wiadomości oraz klucza na bloki
    t = len(message)//32
    Y = '0'
    i = 0
    msglist = wrap(message, 32)
    keylist = wrap(key, 32)
    # Główne operacje zmiennej NH
    while(i < t):
        Y = add_64(Y, multiply_64(add_32(msglist[i+0], keylist[i+0]), add_32(msglist[i+4], keylist[i+4])))
        Y = add_64(Y, multiply_64(add_32(msglist[i+1], keylist[i+1]), add_32(msglist[i+5], keylist[i+5])))
        Y = add_64(Y, multiply_64(add_32(msglist[i+2], keylist[i+2]), add_32(msglist[i+6], keylist[i+6])))
        Y = add_64(Y, multiply_64(add_32(msglist[i+3], keylist[i+3]), add_32(msglist[i+7], keylist[i+7])))
        i += 8
    return Y

def L1HASH(key, message):
    msglist = wrap(message, 8192)
    t = len(msglist)
    
    Len = uint2str(1024*8, 8)
    Y = ''
    # Binarne dodawanie pełnych bloków i operacje funkcji NH
    for i in range(t-1):
        Y += add_64(nh(key, endianswap(msglist[i])), Len)
    # Operacje na niepełnym bloku
    Len = uint2str(len(msglist[t-1]), 8)
    Y += add_64(nh(key, endianswap(zeropad(msglist[t-1], 32))), Len)
    return Y

## Warstwa 2
Druga warstwa haszowania wykorzystuje funkcję wielomianową do haszowania wiadomości. W przypadku gdy wiadomość jest krótsza niż 1024 bajty można pominąć ten krok.<br><br>

<em><b>L2_Hashing</em></b>:
1. Generowane są podklucze 128 i 64 bitowe na podstawie operacji <em><b>AND</em></b> dokonanej na stałych i oryginalnym kluczu.
2. Jeśli wiadomość ma mniej niż 2^17 bajtów wywoływana jest funkcja <em><b>POLY</em></b> dla 64 bitowej liczby pierwszej, w przeciwnym wypadku pierwsze 2^17 bajtów jest hashowane 64 bitowej liczby pierwszej a pozostałe dla 128 bitowej. <br><br>

<em><b>POLY</em></b>:<br>
1. Funkcja przyjmuje zmienne:<br> <em>wordbits</em> - 64 albo 128 to liczba bitów z których składa się liczba pierwsza <br> <em>maxwordrange</em> - maksymalna długość bloku wiadomości <br> <em>key</em> - klucz wykorzystywany w haszowaniu <br> <em>message</em> - haszowana wiadomość<br><br>Na początku tworzone są zmienne pomocnicze:<br> <em>p</em> - liczba pierwsza o długości wordbits<br> <em>offset</em> - przyjmuje wartość 2^wordbis - p<br> <em>marker</em> - przyjmuje wartość p-1
2. Dzieli wiadomość na bytelength(message)/wordbytes bloków.
3. Każdy blok porównywany jest z <em>maxwordrange</em> jeśli jest większy wtedy zarówno <em>marker</em> jak i (blok-offset) są haszowane w przeciwnym wypadku haszowany jest tylko blok.

In [8]:
def poly(wordbits, maxwordrange, key, message):
    # Liczby pierwsze 36, 64 i 128 bitowe  zostały wygenerowane wcześniej by skrócić
    # czas wykonywania programu
    # Przygotowanie zmiennych pomocniczych
    wordbytes = wordbits//8
    if(wordbits == 64):
        p = 10356105370680074483 #prime(2^64)
    elif(wordbits == 128):
        p = 305147961542461497465036432696198208807 #prime(2^128)
    else:
        print("Error")
    offset = 2**wordbits - p
    marker = p-1
    # Podział wiadomości na bloki
    n = (len(message)//8)//wordbytes
    msglist = wrap(message, wordbytes)
    y = 1
    
    for i in range(n):
        m = str2uint(msglist[i])
        # Blok przekracza maxwordrange
        if(m >= maxwordrange):
            y = (key*y+marker)%p
            y = (key*y+(m-offset))%p
        # Pozostałe przypadki
        else:
            y = (key*y+m)%p

    return y

def L2HASH(key, message):
    # Generowanie podkluczy przy pomocy stałych
    Mask64 = uint2str(144115183814443007, 8)
    Mask128 = uint2str(2658455912960639232883337774571192319, 16)
    k64 = str2uint(AND(key[:64], Mask64))
    k128 = str2uint(AND(key[64:], Mask128))
    
    # Haszowanie dla wiadomości krótszej niż 2^17 bajtów
    if(math.ceil((len(message)//8)) <= 2**17):
        y = poly(64, 2**64 - 2**32, k64, message)
    # Haszowanie dla dłuższych wiadomości
    else:
        M1 = message[:2**17]
        M2 = message[2**17:]
        M2 = zeropad(M2+uint2str(128,1), 16)
        y = poly(64, 2**64 - 2**32, k64, M1)
        y = poly(128, 2^128-2^96, k128, uint2str(y, 16) + M2)
    
    Y = uint2str(y, 16)

    return Y

## Warstwa 3
Trzecia warstwa wykorzystuje dwa podklucze jeden 64 bajtowy, drugi 4 bajtowy. Haszuje 16 bajtowy wynik drugiej warstwy do 4 bajtów.
1. Konwertuje wiadomość i pierwszy podklucz na 8 bloków i zmienia je na liczby całkowite.
2. Wiadomość jest haszowana przy pomocy modulo 36-bitowej liczby pierwszej oraz 2^32.
3. Na koniec dokonywana jest operacja <em><b>XOR</em></b> zhaszowanej wiadomości i drugim podkluczu.

In [9]:
def L3HASH(key1, key2, message):
    y = 0
    mlist = []
    k1list = []
    # Zmiana na listę liczb całkowitych 
    for i in range(8):
        mlist.append(str2uint(message[(i*2+1):((i-1)*2)]))
        k1list.append(str2uint(key1[(i*8+1):((i-1)*8)])%68719476731) #prime(2^36)
    for i in range(8):
        y += mlist[i]*k1list[i]
    # Haszowanie liczb całkowitych
    y = y%68719476731 #prime(2^36)
    y = y%(2**32)
    Y = uint2str(y, 4)
    Y = XOR(Y, key2)
    return Y

## Haszowanie uniwersalne
Funkcja zwraca tag o podanej długości (4, 8, 12 lub 16 bajtów). Najpierw generuje 4 podklucze przy pomocy funkcji <em><b>KDF</em></b>. Po jednym dla warstwy pierwszej i drugiej oraz dwa dla warstwy trzeciej. W przypadku gdy wiadomość jest krótsza niż 1024 bajty, warstwa druga jest pomijana.

In [10]:
def UHASH(key, message, taglen):
    iters = int(taglen / 4)
    Y = ''
    # Generacja podkluczy
    L1Key = kdf(key, 1, 1024+(iters-1)*16)
    L2Key = kdf(key, 2, iters*24)
    L3Key1 = kdf(key, 3, iters*64)
    L3Key2 = kdf(key, 4, iters*4)
    for i in range(1, iters+1):
        L1Key_i = L1Key[((i-1)*16)*8:((i-1)*16+1024)*8]
        L2Key_i = L2Key[((i-1)*24)*8:(i*24)*8]
        L3Key1_i = L3Key1[((i-1)*64)*8:(i*64)*8]
        L3Key2_i = L3Key2[((i-1)*4)*8:(i*4)*8]

        A = L1HASH(L1Key_i, message)
        if(len(message) <= len(L1Key_i)):
            B = 64*'0' + A
        else:
            B = L2HASH(L2Key_i, A)
        C = L3HASH(L3Key1_i, L3Key2_i, B)
        Y = Y + C
    return Y

## Algorytm UMAC
1. Na początku generowany jest skrót wiadomości przy pomocy klucza.
2. Generowane jest dopełnienie przy użyciu klucza i wektora nonce.
3. Na skrócie i dopełnieniu wykonywana jest operacja <em><b>XOR</em></b>, wynik działania jest sumą kontrolną.

In [11]:
def UMAC(K, M, nonce, taglen):
    
    HashedMessage = UHASH(K, M, taglen)
    
    Pad = pdf(K, nonce, taglen)
    
    Tag = XOR(Pad, HashedMessage)
    
    return Tag

# blok 632-bitowy 
bin_plaintext = "01001011011100100111100101110000011101000110111101100111011100100110000101100110011010010110000100100000010010110110111101100100011110010010000001100001011101010111010001101111011100100111100101111010011000010110001101101010011010010010000001110111011010010110000101100100011011110110110101101111011100110110001101101001001011000010000001110111011110010110101101101111011100100111101001111001011100110111010001110101011010100110000101100011011001010010000001110101011011100110100101110111011001010111001001110011011000010110110001101110011001010010000001101000011000010111001101111010011011110111011101100001011011100110100101100101"
# klucz 128-bitowy 
bin_key = "00000000000001111111100010001111111110001000111111111000100011111111100011111111000100011111111100010001111111110001000111111111"
# wektor nonce 128-bitowy
nonce = '01001000010000000100110101100011010100010110011001010100011010100101011101101110010110100111001000110100011101010011011101111000'
print("Wiadomość: ", bin_plaintext, '\n')
print("Klucz: ", bin_key, '\n')
print("Wektor Nonce: ", nonce, '\n')
print("Suma kontrolna: ", UHASH(bin_key, bin_plaintext, 16))



Wiadomość:  01001011011100100111100101110000011101000110111101100111011100100110000101100110011010010110000100100000010010110110111101100100011110010010000001100001011101010111010001101111011100100111100101111010011000010110001101101010011010010010000001110111011010010110000101100100011011110110110101101111011100110110001101101001001011000010000001110111011110010110101101101111011100100111101001111001011100110111010001110101011010100110000101100011011001010010000001110101011011100110100101110111011001010111001001110011011000010110110001101110011001010010000001101000011000010111001101111010011011110111011101100001011011100110100101100101 

Klucz:  00000000000001111111100010001111111110001000111111111000100011111111100011111111000100011111111100010001111111110001000111111111 

Wektor Nonce:  01001000010000000100110101100011010100010110011001010100011010100101011101101110010110100111001000110100011101010011011101111000 

Suma kontrolna:  101011001100100000110111011110100100110100001001011

# VMAC

VMAC powstaje poprzez konkatenację Hashu z wiadomości i pseudolosowego paddingu.<br>
### <em>Tag = H_K(M) + F_K(Nonce)</em>
- Tag: Rezultat algorytmu VMAC
- H: Szybka i uniwersalna funkcja Hashująca VHASH
- K: Sekretny klucz
- M: Wiadomość wejściowa
- F: Funkcja generująca pseudolosowy padding
- Nonce: Jednorazowa wartość

VMAC osiąga najlepszą wydajność na systemach 64bitowych, na systemach 32bitowych działa zauważalnie wolniej

<div><img src='resources/VMAC.png' width=700></div>

#### Bezpieczeństwo VMAC
Bezpieczeństwo VMAC w dużej mierze polega na bezpieczeństwu funkcji szyfrującej F oraz na małym prawdopodobieństwie kolizji skrótów.<br>Należy również pamiętać, że wielokrotne użycie tej samej pary (Klucz, Nonce) jest poważnym naruszeniem bezpieczeństwa. Wartosć Nonce należy zmieniać z każdym wyliczonym skrótem. Poznanie dwóch tagów VMAC utworzonych przy użyciu tych samych kluczy i Nonce, pozwala z łatwością fabrykować inne tagi VMAC.

##### Funkcje pomocnicze dla VMAC

Funkcja <em><b>str_to_int</em></b> zamienia string złożony z '0' i '1' na jego reprezentację w systemie dziesiętnym.<br><br>
Funkcja <em><b>int_to_str</em></b> zamienia liczbę na jej reprezentację bitową. Wynik jest zwracany na dowolnej liczbie bitów.<br><br>
Funkcje <em><b>str_add_64, str_add_128 i str_mult_128</em></b> to odpowiednio, zdefiniowane w dokumentacji działania +_64, +_128 i *_128. Zamieniają one ciągi bitowe na liczby i odpowiednio je dodają lub mnożą. Następnie wynik jest zwracany w postaci bitowej na 64/128 bitach.<br><br>
Funkcja <em><b>ZERO_PAD</em></b> dopełnia nam podaną wiadomość zerami do najbliższej wielokrotności wartości <em>nearest_multiple</em>

In [12]:
import math
import AES
import random

def str_to_int(string): # Funkcja zamieniająca ciąg zer i jedynek na jego reprezentację liczbową w Base10
    t = len(string)
    integer = 0
    for i in range(t):
        integer += int(string[i])*pow(2,t-i-1)

    return integer

def int_to_str(integer, length): # Funkcja zamieniająca liczbę Base10 na ciąg zer i jedynek o długości "length"
    if integer==0:
        return '0'*length
    reszty = []
    while(integer!=1):
        reszta = integer%2
        reszty.append(str(reszta))
        integer=integer//2
    reszty.append('1')
    while(len(reszty)<length):
        reszty.append('0')
    reszty.reverse()

    return "".join(reszty)

def str_add_64(string, to_add): # Zdefiniowana w dokumentacji operacja 'string +_64 to_add'
    return int_to_str((str_to_int(string) + str_to_int(to_add))%pow(2,64),64)

def str_add_128(string, to_add): # Zdefiniowana w dokumentacji operacja 'string +_128 to_add'
    return int_to_str((str_to_int(string) + str_to_int(to_add))%pow(2,128),128)

def str_mult_128(string, to_multiply): # Zdefiniowana w dokumentacji operacja 'string *_128 to_multiply'
    return int_to_str((str_to_int(string) * str_to_int(to_multiply))%pow(2,128),128)

def ZERO_PAD(message, nearest_multiple): # Funkcja dopełniająca ciąg zerami do długości równej najbliższej wielokrotności nearest_multiple 

    message_length = len(message)
    n=0
    while(n*nearest_multiple<message_length):
        n+=1

    target_length = n*nearest_multiple
    while(len(message)<target_length):
        message+='0'

    return message

def ZEROS(length): # Funkcja zwracająca ciąg 0 o długości length
    return "0"*length

# message = '100100111010011000110100001010101000101001001101001010101010010001101000111110111100000100101100001001010111011010100000011001001010010100000011101001010011'
# key = '11001011000000110101010010011100001010111001000001110000010010110100110110001010010110101110010101011101110110111000001100011110'
# nonce = '0110010001000010111011000010000110101000111000001011010000100000'

message = generate_random_binary(BLOCK_SIZE*10)
key = generate_random_binary(BLOCK_SIZE)
nonce = generate_random_binary(64)

BLOCK_LENGTH = 128 # Długośc bloku, w przypadku używanego AES jest to 128 bitów
L1KEYLENGTH = 1024

key_length = len(key) #Używany AES używa klucza 128-bitowego
message_length = len(message) # Długośc max 2^64, w przypadku tej implementacji, wielokrotnośc 64-bitów.
nonce_length = len(nonce) # Długość mniejsza niż długość bloku, w naszym przypadku długość bloku AESa
tag_length = 128 # Długość wyjściowego skrótu

##### Funkcje KDF, PDF i NH

Funkcja <em><b>KDF</em></b>:
- Zamienia zmienną <em>index</em> i <em>i</em> na ich reprezentacje bitową o długości odpowiednio 8 oraz <em>BLOCK-LENGTH</em>-8, czyli w naszym przypadku 120, a następnie je konkatenuje ze sobą
- Wynik konkatenacji jest szyfrowany przy użyciu szyfru, w naszym przypadku jest to AES i zwracany

Funkcja <em><b>PDF</em></b>:
- <em>i</em> jest najmniejszą liczbą dla której <em>BLOCK_LENGTH</em>/<em>TAG_LENGTH</em> jest mniejsze, bądź równe 2^i
- Nonce jest zamieniany na liczbę w systemie dziesiętnym i poddawany operacji modulo 2^i
- Z oryginalnego Nonce i ostatnich bitów jest zamieniane na zera
- Nonce jest poprzedzane zerami aby osiągnąć długość bloku, w naszym przypadku 128 bitów
- Następnie AES szyfruje Nonce i zwracany jest odpowiedni fragment szyfrogramu o długości <em>Tag_length</em>

Funkcja <em><b>NH</em></b>:
- Wiadomość i klucz dzielone są na fragmenty o długości 64 bitów
- Dla i-tych i i+1-ych bloków wykonywana jest operacja: 
- <em>NH(i) = NH(i-2) +_128 ((M[i] +_64 K[i]) *_128 (M[i+1] +_64 K[i+1]))</em>
- Pierwsze dwa bity rezultatu są zamieniane na zera

In [13]:
def KDF(key, index, numbits): # Key-derivation function zwraca pseudolosowy ciąg o długości numbits

    global BLOCK_LENGTH
    global L1KEYLENGTH

    n = math.ceil(numbits/BLOCK_LENGTH)

    KDF_result = ''
    for i in range(n):
        index_string = int_to_str(index,8)
        i_string = int_to_str(i,BLOCK_LENGTH-8)
        T = index_string + i_string
        KDF_result += AES.AES_encrypt(T, key)

    KDF_result = KDF_result[0:numbits]

    return KDF_result

def PDF(key, nonce, tag_length): # Pad-derivation function tworzy Pad na podstawie klucza i nonce, który używany jest w Tagu

    global BLOCK_LENGTH

    i = 0
    while(BLOCK_LENGTH/tag_length>pow(2,i)):
        i+=1
    index = str_to_int(nonce)%pow(2,i)
    nonce = nonce[0:len(nonce)-i] + ZEROS(i)
    nonce = ZEROS(BLOCK_LENGTH - len(nonce)) + nonce

    pre_result = AES.AES_encrypt(nonce, key)
    PDF_result = pre_result[index*tag_length:index*tag_length+tag_length]

    return PDF_result

def NH(key, message):

    t = int(len(message)/64)
    chunked_message = [message[i:i+64] for i in range(0, len(message),64)]
    chunked_key = [key[i:i+64] for i in range(0, len(key),64)]
    
    NH_result = '0' * 128
    i = 0

    while i<t:
        NH_result = str_add_128(NH_result,str_mult_128(str_add_64(chunked_message[i],chunked_key[i]),str_add_64(chunked_message[i+1],chunked_key[i+1])))
        i+=2

    two_zeros = '00'
    NH_result = two_zeros + NH_result[2:128]

    return NH_result

##### Funkcje hashujące L1, L2 i L3
Funkcja Hashująca warstwy pierwszej:
- Tworzy pochodną klucza za pomocą <em><b>Key-derivation Function</em></b>
- Dzieli wiadomość na fragmenty o długości <em>L1KEYLENGTH</em>
- Każdy fragment wiadomości jest uzupełniany zerami do najbliższej wielokrotności 128
- Każdy fragment wiadomości, razem z pochodną klucza, jest poddawany działaniu funkcji <em><b>NH</em></b>

Funkcja Hashująca warstwy drugiej:
- Tworzy pochodną klucza za pomocą <em><b>Key-derivation Function</em></b>
- W zależności od iteracji Funkcji Hahsującej warstwy drugiej tworzy podklucz rundy
- Podklucz jest w różnych miejscach wypełniany zerami
- Podklucz jest zapisywany w postaci liczby w systemie dziesiętnym
- Wiadmomość jest dzielona na fragmenty o długości 128 bitów, a następnie zapisywana w postaci dziesiętnej
- <em>Wynik = Wynik * Podklucz + Wiadomość[i]</em> gdzie <em>i</em> jest blokiem dla którego wykonywana jest operacja, a wynik dla i=0 jest równy 1.
- Wynik następnie jest poddawany operacji modulo 2^127-1
- <em>Wynik = Wynik + ((Długość_Wiadomości%L1KEYLENGTH) * 2^64)%(2^127-1)</em>
- Wynik jest przedstawiany w postaci binarnej, na 128 bitach

Funkcja Hashująca warstwy trzeciej:
- Na podstawie <em><b>Key-derivation Function</em></b> i w zależności od rundy, tworzy dwa podklucze i zapisuje je dziesiętnie.
- Na podstawie wiadomości zapisanych dziesiętnie tworzy dwie liczby: <em>Liczba_1 = Wiadomość//(2^64-2^32), Liczba_2 = Wiadomość%(2^64-2^32)</em>
- <em>Wynik = ((Liczba_1 + Podklucz_1)*(Liczba_2 + Podklucz_2))%(2^64-257)</em>
- Wynik jest przedstawiany w postaci binarnej, na 64 bitach


In [14]:
def L1_HASH(key, message, iteration):
    global L1KEYLENGTH
    T = KDF(key, 128, L1KEYLENGTH + 128 * iteration)
    key = T[128 * iteration:L1KEYLENGTH + 128 * iteration]
    L1_HASH_result = ''

    if len(message)>0:
        t = math.ceil(len(message)/L1KEYLENGTH)
        chunked_message = [message[i:i+L1KEYLENGTH] for i in range(0, len(message),L1KEYLENGTH)]
        for i in range(t):
            chunked_message[i] = ZERO_PAD(chunked_message[i], 128)
            L1_HASH_result += NH(key, chunked_message[i])

    return L1_HASH_result

def L2_HASH(key, message, length, iteration):

    subkey = KDF(key, 192, 128 * (iteration + 1))
    subkey = subkey[128 * iteration:128 * (iteration + 1)]
    temp_string = ZEROS(3) + subkey[3:32] + ZEROS(3) + subkey[35:64] + ZEROS(3) + subkey[67:96] + ZEROS(3) + subkey[99:128]
    key_temp = str_to_int(temp_string)

    n = int(len(message)/128)

    if n>0:
        chunked_message = [message[i:i+128] for i in range(0, len(message),128)]
        p127 = pow(2,127) - 1
        pre_result = 1

        for i in range(n):
            chunk_to_int = str_to_int(chunked_message[i])
            pre_result = (pre_result*key_temp + chunk_to_int)%p127
    else:
        pre_result = key_temp

    pre_result = (pre_result + (length%L1KEYLENGTH)*pow(2,64))%p127
    L2_HASH_result = int_to_str(pre_result,128)

    return L2_HASH_result

def L3_HASH(key, message, iteration):

    p64 = pow(2,64) - 257
    i = 0

    need = iteration + 1
    while(need!=0):
        subkey = KDF(key,224,128*(i+1))
        subkey = subkey[128*i:128*(i+1)]
        key_1 = str_to_int(subkey[0:64])
        key_2 = str_to_int(subkey[64:128])
        i+=1
        if key_1 < p64 and key_2 < p64:
            need-=1

    message_1 = str_to_int(message)//(pow(2,64)-pow(2,32))
    message_2 = str_to_int(message)%(pow(2,64)-pow(2,32))
    pre_result = ((message_1 + key_1)*(message_2 + key_2))%p64
    L3_HASH_result = int_to_str(pre_result,64)

    return L3_HASH_result

##### Funkcja VHASH i właściwa funkcja hashująca VHASH
Funkcja <em><b>VHASH</em></b> dla każdych 64 bitów wyjściowej długości tagu wykonuje 3 warstwy hashowania.<br>
Funkcja <em><b>VMAC</em></b> łączy wszystkie poprzednie funkcje i wykonuje właściwe hashowanie zgodne ze standardem VMAC

In [15]:
def VHASH(key, message, tag_length):

    hash = ''
    for i in range(int(tag_length/64)):
        L1_HASH_result = L1_HASH(key, message, i)
        L2_HASH_result = L2_HASH(key, L1_HASH_result, len(message), i)
        hash += L3_HASH(key, L2_HASH_result, i)
    return hash

def VMAC(key, message, nonce, tag_length):

    HashedMessage = VHASH(key, message, tag_length)
    Pad = PDF(key, nonce, tag_length)

    VMAC_result = ''

    for i in range(int(tag_length/64)):
        T = str_add_64(Pad[64*i:64*(i+1)],HashedMessage[64*i:64*(i+1)])
        VMAC_result += T

    return VMAC_result
print("Wiadomość: ", message,"\n")
print("Klucz: ",key,"\n")
print("Nonce:\n",nonce,"\n")
print("Skrót: ", VMAC(key, message, nonce, tag_length))

Wiadomość:  1000111001111101001001010001111010110010011000111101010110001110011111011110110110111101110010000000110000001001011000111101110011111000001011111110111010110110000110010011101001101100010010000011000110011011110100111010100010010001000000111001011001100100111000110111111110010110100100101010001001000101110111111010100001111001000011100011010101110100110100110001111011101010000001110011010111001100100110111000101101001101101110111010001010111011000011111001001110111110011110100000110001000000111000010000001010101100010000000000011000000100110101001110100000111111010101100001100000101001101110110001001011001000101010010110111101101101011111011101010110110001011101001100111010101001011001000000010100010001100001011010101110110100100001101011110101111110010110000010110011001110111101010111111101111100000100101000011111100001111010100000001000010111010111001011010011001001101011001001000010000101010001011000100011011010101100001111100010001000000011011100110111010001111010100001

# Bibliografia
https://datatracker.ietf.org/doc/html/rfc4493 <br>
https://datatracker.ietf.org/doc/html/rfc4418 <br>
https://datatracker.ietf.org/doc/html/rfc2104 <br>
https://datatracker.ietf.org/doc/html/draft-krovetz-vmac-01<br>
https://datatracker.ietf.org/doc/html/rfc3610 <br>

# Autorzy
Jakub Paluch <br>
Jakub Drohomirecki